In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, False, 15)

In [12]:

if file_parameters['watershed'] == 'nak':
    run_num = range(len(folder))
else:
    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2013.xlsx', '나주_2014.xlsx', '나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['용봉_2013.xlsx', '용봉_2014.xlsx', '용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2013.xlsx', '우치_2014.xlsx', '우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2013.xlsx
data/yeong/자동/나주_2014.xlsx
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-6-4 23:00
data/yeong/자동/용봉_2013.xlsx
data/yeong/자동/용봉_2014.xlsx
data/yeong/자동/용봉_2015.xlsx
data/yeong/자동/용봉_2016.xlsx
data/yeong/자동/용봉_2017.xlsx
data/yeong/자동/용봉_2018.xlsx
data/yeong/자동/용봉_2019.xlsx
data/yeong/자동/용봉_2020.xlsx
t

1/1 [==============================] - 0s 20ms/step - gen_loss: 13.3748 - disc_loss: 0.3466 - rmse: 0.4521 - val_loss: 0.4434
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.8487 - disc_loss: 0.3384 - rmse: 0.4304 - val_loss: 0.4780
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.3629 - disc_loss: 0.3629 - rmse: 0.5213 - val_loss: 0.4132
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.8008 - disc_loss: 0.3456 - rmse: 0.4036 - val_loss: 0.5068
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.6488 - disc_loss: 0.3532 - rmse: 0.4781 - val_loss: 0.4085
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5604 - disc_loss: 0.3278 - rmse: 0.4436 - val_loss: 0.3804
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.0543 - disc_loss: 0.3597 - rmse: 0.4124 - val_loss: 0.4299
Epoch 56/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 7.9483 - disc_loss: 0.2715 - rmse: 0.3518 - val_loss: 0.3621
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8735 - disc_loss: 0.2633 - rmse: 0.3828 - val_loss: 0.3417
Epoch 109/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.9555 - disc_loss: 0.2623 - rmse: 0.3163 - val_loss: 0.3362
Epoch 110/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.2146 - disc_loss: 0.2549 - rmse: 0.3262 - val_loss: 0.3452
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0741 - disc_loss: 0.2656 - rmse: 0.3423 - val_loss: 0.3883
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0144 - disc_loss: 0.2595 - rmse: 0.3474 - val_loss: 0.3609
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5117 - disc_loss: 0.2579 - rmse: 0.3498 - val_loss: 0.3417
Epoch 114/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 6.5894 - disc_loss: 0.2352 - rmse: 0.3070 - val_loss: 0.3187
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4662 - disc_loss: 0.2545 - rmse: 0.3789 - val_loss: 0.3589
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6450 - disc_loss: 0.2463 - rmse: 0.3027 - val_loss: 0.3091
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2437 - disc_loss: 0.2354 - rmse: 0.3646 - val_loss: 0.3485
Epoch 169/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.5429 - disc_loss: 0.2499 - rmse: 0.3045 - val_loss: 0.3979
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8150 - disc_loss: 0.2429 - rmse: 0.3060 - val_loss: 0.3716
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2116 - disc_loss: 0.2197 - rmse: 0.3592 - val_loss: 0.3215
Epoch 172/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4137 - disc_loss: 0.2124 - rmse: 0.2869 - val_loss: 0.3124
Epoch 224/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.1988 - disc_loss: 0.2307 - rmse: 0.3387 - val_loss: 0.3217
Epoch 225/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.4867 - disc_loss: 0.2151 - rmse: 0.3187 - val_loss: 0.3635
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4312 - disc_loss: 0.2368 - rmse: 0.3343 - val_loss: 0.3502
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1294 - disc_loss: 0.2285 - rmse: 0.3283 - val_loss: 0.3177
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9256 - disc_loss: 0.2340 - rmse: 0.3220 - val_loss: 0.3373
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8487 - disc_loss: 0.2294 - rmse: 0.3167 - val_loss: 0.3329
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4616 - disc_loss: 0.1878 - rmse: 0.3113 - val_loss: 0.3236
Epoch 514/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0896 - disc_loss: 0.1992 - rmse: 0.3165 - val_loss: 0.2919
Epoch 515/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9742 - disc_loss: 0.2024 - rmse: 0.2643 - val_loss: 0.3520
Epoch 516/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2429 - disc_loss: 0.1979 - rmse: 0.2763 - val_loss: 0.2706
Epoch 517/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3094 - disc_loss: 0.2097 - rmse: 0.2588 - val_loss: 0.3006
Epoch 518/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6345 - disc_loss: 0.1984 - rmse: 0.3272 - val_loss: 0.3400
Epoch 519/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.7259 - disc_loss: 0.2038 - rmse: 0.3191 - val_loss: 0.2777
Epoch 520/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7357 - disc_loss: 0.2072 - rmse: 0.2840 - val_loss: 0.2418
Epoch 572/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2572 - disc_loss: 0.1970 - rmse: 0.2806 - val_loss: 0.3076
Epoch 573/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5144 - disc_loss: 0.1940 - rmse: 0.2972 - val_loss: 0.3134
Epoch 574/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4716 - disc_loss: 0.2019 - rmse: 0.2915 - val_loss: 0.2923
Epoch 575/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0929 - disc_loss: 0.1841 - rmse: 0.2694 - val_loss: 0.3054
Epoch 576/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5582 - disc_loss: 0.1852 - rmse: 0.2709 - val_loss: 0.2463
Epoch 577/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2887 - disc_loss: 0.1851 - rmse: 0.3030 - val_loss: 0.3310
Epoch 578/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9837 - disc_loss: 0.1938 - rmse: 0.2578 - val_loss: 0.2556
Epoch 630/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9628 - disc_loss: 0.1947 - rmse: 0.2722 - val_loss: 0.2829
Epoch 631/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7375 - disc_loss: 0.1930 - rmse: 0.2683 - val_loss: 0.3315
Epoch 632/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0625 - disc_loss: 0.1885 - rmse: 0.2606 - val_loss: 0.3013
Epoch 633/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3534 - disc_loss: 0.1781 - rmse: 0.2933 - val_loss: 0.3560
Epoch 634/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2149 - disc_loss: 0.1891 - rmse: 0.2712 - val_loss: 0.2712
Epoch 635/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8884 - disc_loss: 0.1796 - rmse: 0.2872 - val_loss: 0.2993
Epoch 636/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0007 - disc_loss: 0.1841 - rmse: 0.2796 - val_loss: 0.3227
Epoch 688/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0946 - disc_loss: 0.1983 - rmse: 0.2845 - val_loss: 0.3003
Epoch 689/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7095 - disc_loss: 0.1911 - rmse: 0.2665 - val_loss: 0.2811
Epoch 690/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4316 - disc_loss: 0.1863 - rmse: 0.2539 - val_loss: 0.3282
Epoch 691/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0041 - disc_loss: 0.1931 - rmse: 0.2705 - val_loss: 0.2724
Epoch 692/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3732 - disc_loss: 0.1791 - rmse: 0.2810 - val_loss: 0.3122
Epoch 693/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6407 - disc_loss: 0.1824 - rmse: 0.2769 - val_loss: 0.2852
Epoch 694/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0152 - disc_loss: 0.1793 - rmse: 0.2741 - val_loss: 0.2787
Epoch 746/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4069 - disc_loss: 0.1684 - rmse: 0.2916 - val_loss: 0.3009
Epoch 747/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8517 - disc_loss: 0.1860 - rmse: 0.2467 - val_loss: 0.2869
Epoch 748/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2205 - disc_loss: 0.1815 - rmse: 0.3217 - val_loss: 0.2854
Epoch 749/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4290 - disc_loss: 0.1850 - rmse: 0.2716 - val_loss: 0.2853
Epoch 750/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3203 - disc_loss: 0.1977 - rmse: 0.2754 - val_loss: 0.3090
Epoch 751/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8486 - disc_loss: 0.1847 - rmse: 0.2986 - val_loss: 0.2674
Epoch 752/2000
1/1 [===============

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [94]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'chl-a'
print(rnn_target_column)



(65088, 36)
chl-a


In [95]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (45561, 36) val_df.shape :  (13018, 36) test_df.shape: (6509, 36)


In [96]:
#fake_df.shape[0]/8760

In [97]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [98]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [99]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong/chlorophyll-a/


In [100]:
rnn_epochs, rnn_in_setps

(15, 240)

## 모델 학습

In [101]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    #training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
5/5 [==============================] - 10s 2s/step - loss: 1.1427 - mean_absolute_error: 0.6957 - nse: 0.0265 - val_loss: 1.6229 - val_mean_absolute_error: 0.8506 - val_nse: 0.1442
Epoch 2/15
5/5 [==============================] - 10s 2s/step - loss: 0.8429 - mean_absolute_error: 0.6077 - nse: 0.2758 - val_loss: 1.0326 - val_mean_absolute_error: 0.7001 - val_nse: 0.4553
Epoch 3/15
5/5 [==============================] - 10s 2s/step - loss: 0.6659 - mean_absolute_error: 0.5370 - nse: 0.4329 - val_loss: 0.7578 - val_mean_absolute_error: 0.5948 - val_nse: 0.5991
Epoch 4/15
5/5 [==============================] - 10s 2s/step - loss: 0.5459 - mean_absolute_error: 0.4912 - nse: 0.5141 - val_loss: 0.6954 - val_mean_absolute_error: 0.5686 - val_nse: 0.6299
Epoch 5/15
5/5 [==============================] - 10s 2s/step - loss: 0.4850 - mean_absolute_error: 0.4638 - nse: 0.5571 - val_loss: 0.6496 - val_mean_absolute_error: 0.5549 - val_nse: 0.6535
Epoch 6/15
5/5 [========================

In [102]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/yeong/models/yeong/chlorophyll-a/
year : 2013 ~ 2020
run :  [0]
target :  chl-a   7
length :  65088
train=7, test=1, val=2
all_nse :  [0.59749335]
all_pbias :  [1.8744807]
all_mae :  14.973919
all_rmse :  21.822659
all_Rs :  0.5979841202584026
all_R :  0.7732943


In [103]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.6102963]
train_pbias :  [-1.9907898]
train_mae :  13.7272005
train_rmse :  19.963167
train_Rs :  0.6108132132295054
train_R :  0.7815454


In [104]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

/home/jaewon/work/venv/water-quality-venv/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.10727012]
val_pbias :  [6.3347898]
val_mae :  17.95301
val_rmse :  25.975273
val_Rs :  0.16179999708885706
val_R :  0.4022437


In [105]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.60187256]
test_pbias :  [1.3593493]
test_mae :  18.462124
test_rmse :  26.209436
test_Rs :  0.6022816824776243
test_R :  0.7760681


In [106]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 4,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1
'''
